In [ ]:
from llama_index import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    download_loader,
    GPTVectorStoreIndex,
)
from llama_hub.github_repo import GithubRepositoryReader, GithubClient
import os
import nest_asyncio

nest_asyncio.apply()

In [ ]:
# Fetch GitHub Reader
download_loader("GithubRepositoryReader")
github_client = GithubClient(os.environ["GH_TOKEN"])

# Config Github Reader
loader = GithubRepositoryReader(
    github_client,
    owner="duneanalytics",
    repo="docs",
    filter_directories=(["docs"], GithubRepositoryReader.FilterType.INCLUDE),
    filter_file_extensions=([".md"], GithubRepositoryReader.FilterType.INCLUDE),
    verbose=True,
    concurrent_requests=10,
)

# Load docs from Github
docs = loader.load_data(branch="master")

# Construct a simple vector index
index = GPTVectorStoreIndex.from_documents(docs)

# Saving Index for future use
index.storage_context.persist()

In [ ]:
# Querying the index
query_engine = index.as_query_engine()
response = query_engine.query("what is a Dune wizard?")
print(response)